This Notebook: |  
1) Extracts targeted data about goodreads books from large scraped file and saves to a json file for use elsewhere.
2) Makes a Search Engine for finding specific book records using keywords

**Ultimately these two actions should probably be separated into two different notebooks**

FILES USED IN PROJECT
(All files takes from UCSD Book Graph Website :
https://sites.google.com/eng.ucsd.edu/ucsdbookgraph/home?pli=1 )

goodreads_interactions.csv (~4G) |  
--- 3 columns: user_id, book_id, rating |  
--- int, int, int(0-5) |  
--- https://drive.google.com/open?id=1zmylV7XW2dfQVCLeg1LbllfQtHD2KUon

goodreads_books.json.gz (~2G) |  
--- each line is a json {} containinga alot of book metadata |  
--- https://drive.google.com/uc?id=1LXpK1UfqtP89H1tYy0pBGHjYk8IhigUK

book_id_map.csv |  
--- 2 columns: book_id_csv, book_id |  
--- int, int |  
--- maps between book_id in goodreads_interactions.csv and goodreads_books.jason.gz|  
--- https://drive.google.com/uc?id=1CHTAaNwyzvbi1TR08MJrJ03BxA266Yxr


In [3]:
#DEFINE DATA FILE LOCATIONS (RELATIVE)
bookRecordsFile = "GoodReadsData/goodreads_books.json.gz"

#As of 20221107 the following two files aren't used in this notebook, but I'm keeping them here so easy to find for subsequent notebooks
interactionRecordsFile = "GoodReadsData/goodreads_interactions.csv"
BookIdMapFile_userInteractionToBookRecord = "GoodReadsData/book_id_map.csv"


In [4]:
# CHECK FILE SIZE
# CODING NOTE: ! runs things on command line $ inserts variables into command
fileLines = !wc -l $bookRecordsFile #This command counts number of lines in file (stands for "word count - lines")
print(f"Lines in File:\n{fileLines}")
fileSize = !ls -lh $bookRecordsFile #show file size for all files in directory containing specified name
print(f"File Size Info:\n{fileSize}")

Lines in File:
[' 7588375 GoodReadsData/goodreads_books.json.gz']
File Size Info:
['-rw-r--r--@ 1 karina  staff   1.9G Nov 27 09:30 GoodReadsData/goodreads_books.json.gz']


Conclusion from above file size analysis: json file is too big to handily work with the whole thing, so instead will work with it in a streaming fashion.

In [5]:
# CREATE FUNCTION TO TAKE A BOOK'S FULL RAW DATA AND RETURN JUST THE DATA WE WANT
import json
def parse_fields(line): #Takes a line of json (one line per book) and extracts just the fields we want into a dict for that book
    bookDict = json.loads(line) #turns json line into a dict 
    return { #return dict with just the subset of fields we want 
        "bookRecord_bookId": bookDict["book_id"],
        "title": bookDict["title_without_series"],
        "ratings": bookDict["ratings_count"],
        "url": bookDict["url"],
        "cover_image": bookDict["image_url"]
    }
        

In [6]:
# STREAM IMPORT BOOK RECORDS AND KEEP JUST THE DATA WE NEED
#by streaming file line-by-line and taking only:
#  -- books with >15 ratings
#  -- a subset of fields from each book

import gzip
books_records_listOfDicts = []
with gzip.open(bookRecordsFile,"r") as f: #opens file without unzipping the whole thing upfront... unzipping would take it from ~2GB to ~10
    while True:
        rawBookEntry = f.readline() #stream line by line to not require memory to hold entire file at once
        if not rawBookEntry: #when reaches end of file, line will return none
            break
        prunedBookEntry = parse_fields(rawBookEntry)
        
        try:
            ratings = int(prunedBookEntry["ratings"])
        except ValueError:
            continue #often ratings are missing, continue stops this iteration of the loop and continues to the next
        if ratings > 15:
            books_records_listOfDicts.append(prunedBookEntry)

In [7]:
#CONVERT IMPORTED BOOK RECORD DATA INTO A DATA FRAME AND CLEAN COLUMNS

import pandas as pd
book_records = pd.DataFrame.from_dict(books_records_listOfDicts) #turns list of dicts into a df with one row per dict
book_records["ratings"] = pd.to_numeric(book_records["ratings"]) #makes ratings column numeric rather than eg string
book_records["mod_title"] = book_records["title"].str.replace("[^a-zA-Z0-9 ]","", regex=True) #creates modified titles that deletes anything that isnt a letter, number, or space (eg punctutaion)
book_records["mod_title"] = book_records["mod_title"].str.lower()
book_records["mod_title"] = book_records["mod_title"].str.replace("\s+", " ", regex=True) #replace any mutliple spaces with single space
book_records = book_records[book_records["mod_title"].str.len() > 0] #Get rid of any titles that have nothing left for their modTitle after above modifications


In [8]:
#SAVE CLEANED BOOK RECORD DATA TO JSON FOR USE ELSEWHERE
book_records.to_json("books_titles.json")
book_records.head()

,bookRecord_bookId,title,ratings,url,cover_image,mod_title
0,7327624,"The Unschooled Wizard (Sun Wolf and Starhawk, ...",140,https://www.goodreads.com/book/show/7327624-th...,https://images.gr-assets.com/books/1304100136m...,the unschooled wizard sun wolf and starhawk 12
1,6066819,Best Friends Forever,51184,https://www.goodreads.com/book/show/6066819-be...,https://s.gr-assets.com/assets/nophoto/book/11...,best friends forever
2,287141,The Aeneid for Boys and Girls,46,https://www.goodreads.com/book/show/287141.The...,https://s.gr-assets.com/assets/nophoto/book/11...,the aeneid for boys and girls
3,6066812,All's Fairy in Love and War (Avalon: Web of Ma...,98,https://www.goodreads.com/book/show/6066812-al...,https://images.gr-assets.com/books/1316637798m...,alls fairy in love and war avalon web of magic 8
4,287149,The Devil's Notebook,986,https://www.goodreads.com/book/show/287149.The...,https://images.gr-assets.com/books/1328768789m...,the devils notebook


In [9]:
#MAKE MATRIX TO SEARCH ALL BOOKS IN DATASET

#Uses "term frequency-inverse document frequency" (tfidf) approach
#Every unique word from all titles becomes a column of a matrix
#Each searchable sentence (title) has a row
#Each entry is order of magnitude more often word appears in all sentences vs specific sentence
#Formula: log[(freq(word) all sentences)/(freq(word) this sentence)]

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer() #tfidf analysis object  
tfidf = vectorizer.fit_transform(book_records["mod_title"]) #make tfidf matrix from titles in mod_title column

In [10]:
#MAKE FUNCTION TO SEARCH FOR CLOSEST MATCHES TO A QUERY TITLE 

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re

def make_clickable(URL):
    return f'<a target ="_blank" href="{URL}">Goodreads Listing</a>' #creates html to link to title, which can be used when diplaying book results

def show_image(imageSource):
    return f'<img src="{imageSource}" width=50></img>' #creates html to display image associated with title, which can be used when diplaying book results

def search(query,vectorizer):
    processed_query = re.sub("[^a-zA-Z0-9 ]","",query.lower()) #makes query closer to mod_title format by lowercasing and keeping only letters/numbers/spaces
    query_vec = vectorizer.transform([processed_query]) #turns query into a tfidf vector
    similarity = cosine_similarity(query_vec, tfidf).flatten() #computes cosine similarity with each title and flatten makes it return as a numpy array with one entry per title
    indices = np.argpartition(similarity, -10)[-10:] #find indices of 10 largest similarity values
    results = book_records.iloc[indices] #gets book titles at those indices... at this point can return more than one book per title if there is more than one entry in goodreads
    results = results.sort_values("ratings", ascending = False) #sort by number of ratings for each title
    refinedResults = results.head(30) #only the book of the n most similar title with the highest number of ratings
    return refinedResults.style.format({'url': make_clickable, 'cover_image': show_image}) #adds formating to results so url's are clickable and image of book cover shows

USER ENTRY REQUIRED |   
The following cell is not meant to stay hard coded. Use it to find goodreads book records using title keywords:

In [11]:
KeyWords = "Bet me" #USER ENTRY
search(KeyWords, vectorizer)

,bookRecord_bookId,title,ratings,url,cover_image,mod_title
894388,854757,Bet Me,72038,Goodreads Listing,,bet me
89000,34006007,Bet Me,1581,Goodreads Listing,,bet me
1296906,6491947,Bet Me,1033,Goodreads Listing,,bet me
919808,8880488,Bet Me,160,Goodreads Listing,,bet me
745986,111974,Bet Me,141,Goodreads Listing,,bet me
787243,10001126,Bet Me,86,Goodreads Listing,,bet me
536936,12136174,Bet Me,76,Goodreads Listing,,bet me
959463,161268,Bet Me,28,Goodreads Listing,,bet me
141663,527810,Bet Me,27,Goodreads Listing,,bet me
959464,161269,Bet Me,18,Goodreads Listing,,bet me


USER ENTRY REQUIRED: |  
The following cell is not meant to stay hardcoded. |  
Make a list of liked books using book_ids found using above cell


In [12]:
queried_bookRecord_bookIds = [33571217, 854757] #[33571217, 1140906, 4980, 854757]; 

In [13]:
# CREATE AND SAVE CSV OF LIKED BOOKS FOR LATE USE

#pull data about all liked books
liked_bookRecord_bookIds=[]
liked_titles=[]
liked_user_ids = []
liked_ratings=[]

for ind,row in book_records.iterrows():
    if int(row["bookRecord_bookId"]) in queried_bookRecord_bookIds:
        liked_bookRecord_bookIds.append(int(row["bookRecord_bookId"]))
        liked_titles.append(row["title"])
        liked_user_ids.append(-1) #all the same user. -1 so don't inadvertently use an id matching an extant goodreads user
        liked_ratings.append(5) #assume querier "liking" a book means a rating of 5 for now. Could make this more nuanced in future.

#add all liked data to a dict
liked_book_records = { 
    "user_id" : liked_user_ids,
    "bookRecord_bookId" : liked_bookRecord_bookIds,
    "rating" : liked_ratings,
    "title" : liked_titles
}

#convert dict to df
liked_book_records = pd.DataFrame(liked_book_records)

#save dataframe as csv
liked_book_records.to_csv("liked_books.csv")

In [14]:
liked_book_records.head()

,user_id,bookRecord_bookId,rating,title
0,-1,854757,5,Bet Me
1,-1,33571217,5,Reincarnation Blues
